In [1]:
# =============================================================================
# CELL 1: IMPORTS WITH WINDOWS/VPN FIXES 
# =============================================================================

import sys
import warnings

# Suppress noisy warnings
warnings.filterwarnings('ignore')

# === WINDOWS ASYNC FIX ===
if sys.platform == 'win32':
    import asyncio
    try:
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        print("✅ Windows async policy set")
    except Exception as e:
        print(f"⚠️ Async policy warning: {e}")

# === SSL/VPN FIXES ===
import os
os.environ['PYTHONHTTPSVERIFY'] = '0'
os.environ['CURL_CA_BUNDLE'] = ''

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === MAIN IMPORTS ===
import pandas as pd
import numpy as np
import requests
import io
import json
import time

# === YAHOO IMPORTS (with error handling) ===
try:
    from yahooquery import Ticker
    print("✅ yahooquery loaded")
except Exception as e:
    print(f"❌ yahooquery failed: {e}")
    Ticker = None

try:
    import yfinance as yf
    print("✅ yfinance loaded")
except Exception as e:
    print(f"❌ yfinance failed: {e}")
    yf = None

print("\n🎉 All imports complete!")

✅ Windows async policy set
✅ yahooquery loaded
✅ yfinance loaded

🎉 All imports complete!


In [2]:
# =============================================================================
# CELL 2: CONFIGURATION AND SETUP
# =============================================================================


import os
import json



# Saves all configuration settings and helper functions
DATA_FOLDER = "YfinanceDataDump"

# Check if the folder already exists.
if not os.path.exists(DATA_FOLDER):
    try:
        # os.makedirs() creates the folder
        os.makedirs(DATA_FOLDER)
        print(f"Created data folder: {DATA_FOLDER}")
    except Exception as e:
        # If something goes wrong (like no permission), save to current folder
        print(f"Warning: Could not create folder '{DATA_FOLDER}'. Saving to current directory. Error: {e}")
        DATA_FOLDER = "."  # The dot means "current folder"

# 2. FILE PATHS
# -------------
# These are the names of the files where we'll save our results
# os.path.join() combines the folder name with the file name properly
# (handles / vs \ on different operating systems automatically)
CACHE_FILE = os.path.join(DATA_FOLDER, "financial_cache.json")    # Stores data we've already fetched
FORTRESS_CSV = os.path.join(DATA_FOLDER, "fortress_stocks.csv")   # Best quality stocks
STRONG_CSV = os.path.join(DATA_FOLDER, "strong_stocks.csv")       # Good quality stocks
RISKY_CSV = os.path.join(DATA_FOLDER, "risky_stocks.csv")         # Lower quality stocks
ANALYST_CSV = os.path.join(DATA_FOLDER, "Analyst_Fortress_Picks.csv")  # Analyst favorites
BUFFETT_CSV = os.path.join(DATA_FOLDER, "Buffett_Value_Picks.csv") # Value picks
DEEPVAL_CSV = os.path.join(DATA_FOLDER, "Deep_Value_Gems.csv")    # Intersection of Buffett + Burry
# 3. UNIVERSE FILTERS - Minimum requirements for a stock to be considered

MIN_PRICE = 2.00              # Stock price must be at least $2 (avoid "penny stocks")
MIN_VOLUME = 1_000_000          # At least 1,000,000 shares traded per day (on average)

MIN_CAP = 300_000_000          # Market cap at least $300 million (company value)
                              # Market Cap = Stock Price × Number of Shares
MIN_CURRENT_RATIO = 1.2       # Current Ratio must be > 1.2
                              # Current Ratio = Current Assets / Current Liabilities
                              # If > 1, the company can pay its short-term bills
MAX_PE_RATIO = 100.0          # P/E ratio must be less than 100
                              # P/E = Price / Earnings Per Share
                              

# 4. SAFETY THRESHOLDS - Additional quality filters
# -------------------------------------------------
MIN_INTEREST_COVERAGE = 1.5   # Interest Coverage > 1.5 means company earns 1.5x what it owes in interest
                              # Interest Coverage = EBIT / Interest Expense
                              # If < 1, company can't pay its interest bills = bad!
MIN_ROIC = 0.05               # ROIC (Return on Invested Capital) > 5%
                              # ROIC = Operating Profit / Capital Invested
                              # Shows how well management uses money to generate returns
FORTRESS_MARGIN_THRESHOLD = 0.05  # Operating Margin > 5% for "Fortress" (best) tier
                                   # Operating Margin = Operating Income / Revenue
                                   # Shows what % of sales becomes actual profit

# Sectors we want to EXCLUDE (skip)
# Financial Services and Real Estate have very different financial structures
EXCLUDED_SECTORS = ['Financial Services', 'Real Estate']

CACHE_EXPIRY_DAYS = 30  # Re-fetch data if our saved data is older than 30 days


# ==========================================
# HELPER FUNCTIONS - Reusable code blocks
# ==========================================


def load_cache():
    """
    Load previously saved financial data from our cache file.
    Returns:
        dict: A dictionary of saved data, or empty dict {} if no cache exists
    """
    # Check if the cache file exists
    if os.path.exists(CACHE_FILE):
        try:
            # 'r' means "read mode" - we're reading the file, not writing to it
            with open(CACHE_FILE, 'r') as f:
                # json.load() reads the JSON file and converts it to a Python dictionary
                return json.load(f)
        except:
            # If something goes wrong reading the file, return empty dictionary
            return {}
    # If file doesn't exist, return empty dictionary
    return {}

def save_cache(cache_data):
    """
    Save our financial data to a file so we can reuse it later.
    
    Args:
        cache_data (dict): The data we want to save
    """
    try:
        # 'w' means "write mode" - creates the file or overwrites existing
        with open(CACHE_FILE, 'w') as f:
            # json.dump() converts the Python dictionary to JSON format and saves it
            json.dump(cache_data, f)
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")

def calculate_altman_z_yfinance(bs, fin, market_cap):
    """
    Calculate the Altman Z-Score - a formula that predicts bankruptcy risk.
    
    Created by Professor Edward Altman in 1968.
    - Z > 2.99: "Safe Zone" - company is financially healthy
    - Z between 1.81 and 2.99: "Grey Zone" - uncertain
    - Z < 1.81: "Distress Zone" - high risk of bankruptcy
    
    The Formula: Z = 1.2A + 1.4B + 3.3C + 0.6D + 1.0E
    
    Where:
    A = Working Capital / Total Assets (liquidity)
    B = Retained Earnings / Total Assets (profitability over time)
    C = EBIT / Total Assets (operating efficiency)
    D = Market Value of Equity / Total Liabilities (market confidence)
    E = Sales / Total Assets (asset efficiency)
    
    Args:
        bs: Balance Sheet data (DataFrame from yfinance)
        fin: Financial data (DataFrame from yfinance)  
        market_cap: Market capitalization in dollars
        
    Returns:
        float: The Z-Score, or 0 if calculation fails
    """
    try:
        # Helper function to safely get values from DataFrames
        # Sometimes the data has different names, so we try multiple
        def get_val(df, keys):
            for k in keys:
                if k in df.index:
                    # .iloc[0] gets the first value (most recent year)
                    return df.loc[k].iloc[0]
            return 0

        # Get values from Balance Sheet (bs) and Financials (fin)
        total_assets = get_val(bs, ['Total Assets'])
        total_liab = get_val(bs, ['Total Liabilities Net Minority Interest', 'Total Liabilities'])
        current_assets = get_val(bs, ['Current Assets', 'Total Current Assets'])
        current_liab = get_val(bs, ['Current Liabilities', 'Total Current Liabilities'])
        retained_earnings = get_val(bs, ['Retained Earnings'])
        
        ebit = get_val(fin, ['EBIT', 'Operating Income'])  # Earnings Before Interest & Taxes
        total_revenue = get_val(fin, ['Total Revenue'])
        
        # Can't divide by zero, so return 0 if missing key data
        if total_assets == 0 or total_liab == 0: 
            return 0

        # Calculate each component of the Z-Score formula
        # A: Working Capital (can company pay short-term bills?)
        A = (current_assets - current_liab) / total_assets
        
        # B: Retained Earnings (accumulated profits over the years)
        B = retained_earnings / total_assets
        
        # C: EBIT (operating profitability)
        C = ebit / total_assets
        
        # D: Market Cap vs Debt (how much does market trust vs owe?)
        D = market_cap / total_liab
        
        # E: Revenue (is the company using its assets efficiently?)
        E = total_revenue / total_assets
        
        # The final formula with Altman's coefficients
        return (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
    
    except Exception as e:
        return 0  # Return 0 if any calculation fails

In [3]:
# =============================================================================
# CELL 3: STEP 1 - FETCH US STOCK UNIVERSE
# =============================================================================


def get_combined_universe():
    """
    Get a list of all US stock tickers from NASDAQ.
    
    The NASDAQ provides a file with all traded stocks.
    US tickers don't need a suffix (like .TO for Toronto).
    
    Returns:
        list: List of US stock symbols like ['AAPL', 'MSFT', 'GOOGL', ...]
    """
    print("--- STEP 1: Fetching North American Universe ---")
    tickers = []
    
    # Fetch from NASDAQ's official list
    try:
        url_us = "https://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        
        # Download the file
        s = requests.get(url_us).content
        
        # Read it as a CSV with pipe (|) as the separator
        # The file looks like: Symbol|Security Name|ETF|Test Issue|...
        df_us = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')
        
        # Filter out test issues and ETFs (we only want real stocks)
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        
        # Get symbols, clean them, and keep only short ones (< 5 characters)
        # Long symbols are usually special securities we don't want
        us_list = [x.replace('$', '-') for x in df_us['Symbol'].astype(str) if len(x) < 5]
        
        tickers.extend(us_list)  # Add to our list
        print(f"   -> Found {len(us_list)} US stocks.")
        
    except:
        print("   -> Error fetching USA list.")

    return tickers

In [4]:
# =============================================================================
# CELL 4: STEP 2 - LIGHTWEIGHT FILTER (QUICK SCREENING)
# =============================================================================
# This is the first filter - it quickly eliminates stocks that don't meet
# basic requirements. It's "lightweight" because it uses fast bulk data fetching.

def get_initial_survivors(tickers):
    """
    Filter stocks using basic criteria. 
    Args:
        tickers (list): List of all stock symbols to check
        
    Returns:
        DataFrame: Table of stocks that passed all filters with their metrics
    """
    print(f"\n--- STEP 2: Running 'Lightweight' Filter on {len(tickers)} stocks ---")
    
    chunk_size = 500  # Process stocks in batches of 500 at a time
    survivors = []    # This list will hold stocks that pass all filters
    
    # Split our list into chunks (batches) divides a large request into smaller parts
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    # Process each chunk
    for i, chunk in enumerate(chunks):
        # Print progress every 5 batches
        if i % 5 == 0: 
            print(f" -> Processing Batch {i+1}/{len(chunks)}...")
        
        try:
            # Create a Ticker object for all stocks in this chunk
            # asynchronous=True means it fetches data for multiple stocks simultaneously
            yq = Ticker(chunk, asynchronous=True)
            
            # Get multiple types of data at once (this is the "bulk fetch")
            # These are different Yahoo Finance data modules
            df_modules = yq.get_modules(
                'summaryProfile summaryDetail financialData price defaultKeyStatistics'
            )
            
            # Loop through each stock's data
            for symbol, data in df_modules.items():
                # If data is just a string, it means there was an error
                if isinstance(data, str): 
                    continue
                
                try:
                    # Extract the stock price
                    # The .get() method returns the value if it exists, or 0 if it doesn't
                    price = data.get('price', {}).get('regularMarketPrice', 0)
                    if price is None: 
                        price = 0
                    
                    # Extract average daily trading volume
                    vol = data.get('summaryDetail', {}).get('averageVolume', 0)
                    if vol is None or vol == 0:
                        # Try alternative location for volume data
                        vol = data.get('price', {}).get('averageDailyVolume10Day', 0)
                    
                    # Extract market capitalization (total company value)
                    cap = data.get('price', {}).get('marketCap', 0)
                    if cap is None: 
                        cap = 0
                    
                    # Extract sector (Technology, Healthcare, etc.)
                    sector = data.get('summaryProfile', {}).get('sector', 'Unknown')
                    
                    # Get financial data
                    fin_data = data.get('financialData', {})
                    curr_ratio = fin_data.get('currentRatio', 0)      # Current Ratio
                    op_margins = fin_data.get('operatingMargins', 0)  # Operating Margin (as decimal)
                    if curr_ratio is None: 
                        curr_ratio = 0
                    if op_margins is None: 
                        op_margins = 0

                    # Get P/E ratio
                    pe = data.get('summaryDetail', {}).get('trailingPE')

                    # ====== APPLY FILTERS ======
                    # Each 'continue' statement skips to the next stock (odd yes but continue skipping to the next stock essentially means it failed the filter and we move on)
                    
                    # Skip if P/E is too high (overvalued)
                    if pe is not None and pe > MAX_PE_RATIO: 
                        continue
                    
                    # Skip if price is too low (penny stock)
                    if price < MIN_PRICE: 
                        continue
                    
                    # Skip if company is too small
                    if cap < MIN_CAP: 
                        continue
                    
                    # Skip if not enough trading volume (hard to buy/sell)
                    if vol < MIN_VOLUME: 
                        continue
                    
                    # Skip if in excluded sectors
                    # any() returns True if ANY item in the list is True
                    if any(x in sector for x in EXCLUDED_SECTORS): 
                        continue
                    
                    # Skip if current ratio is too low (can't pay bills)
                    if curr_ratio < MIN_CURRENT_RATIO: 
                        continue
                    
                    # Skip if operating margin is zero or negative (not profitable)
                    if op_margins <= 0: 
                        continue
                    
                    # If we get here, the stock passed ALL filters!
                    # Add it to our survivors list
                    survivors.append({
                        'Ticker': symbol,
                        'Sector': sector,
                        'Price': price,
                        'Op Margin %': round(op_margins * 100, 2),  # Convert to percentage
                        'P/E': round(pe, 2) if pe else 0,
                        'Curr Ratio': curr_ratio,
                        'Mkt Cap (B)': round(cap / 1_000_000_000, 2)  # Convert to billions
                    })
                    
                except:
                    continue  # Skip this stock if any error occurs
                    
        except:
            continue  # Skip this batch if any error occurs
    
    # Convert our list of dictionaries to a pandas DataFrame (table)
    return pd.DataFrame(survivors)

In [5]:
# =============================================================================
# CELL 5: STEP 3 - DEEP FINANCIAL ANALYSIS
# =============================================================================
# This is the detailed analysis phase. For each stock that passed Step 2,
# we download detailed financial statements and calculate advanced metrics.

def get_advanced_metrics(survivor_df):
    """
    Perform deep financial analysis on stocks that passed initial screening.

    
    Args:
        survivor_df: DataFrame of stocks from Step 2
        
    Returns:
        DataFrame: Stocks with full analysis and tier classification
    """
    tickers = survivor_df['Ticker'].tolist()
    print(f"\n--- STEP 3: Fetching Deep Financials for {len(tickers)} Survivors ---")
    
    # Load our cached data (data we've already fetched before)
    cache = load_cache()
    current_time = time.time()  # Current time in seconds since 1970 (Unix timestamp)
    expiry_seconds = CACHE_EXPIRY_DAYS * 86400  # Convert days to seconds (86400 sec/day)
    
    final_data = []  # Will store our results
    
    # Loop through each stock
    for i, ticker in enumerate(tickers):
        # Print progress every 20 stocks
        if i % 20 == 0: 
            print(f" -> Analyzing {i+1}/{len(tickers)}: {ticker}...")
        
        # Uncomment the line below if you're getting throttled (too many requests)
        # time.sleep(0.75)  # Wait 0.75 seconds between requests
        
        def determine_tier_history(metrics, is_fortress_margin, is_pos_margin):
            """
            Determine tier based on Margins AND Financial Health (Z-Score).
            
            STRICT RULES:
            1. Fortress = High Margin (>5%) AND Safe Z-Score (>2.99)
            2. Strong   = Positive Margin (>0%) AND Acceptable Z-Score (>1.81)
            3. Risky    = Fails either margins or safety
            """
            
            # Extract Z-Score from the metrics dictionary
            z_val = metrics.get('z_score', 0)
            
            # 1. IMMEDIATE FAILURES (Hard Safety Stops)
            # If a company can't pay interest or has low return on capital, it's Risky.
            if metrics['int_cov'] < MIN_INTEREST_COVERAGE: 
                return "Risky"
            if metrics['roic'] < MIN_ROIC: 
                return "Risky"
            
            # 2. FORTRESS CRITERIA (The "Perfect" Stock)
            # Must have BOTH strong margins AND a safe Z-Score
            if is_fortress_margin and z_val >= 2.99:
                return "Fortress"
                
            # 3. STRONG CRITERIA (The "Good" Stock)
            # Must have at least positive margins AND be out of the "Distress Zone"
            # We use 1.81 because Z < 1.81 implies high bankruptcy risk
            elif is_pos_margin and z_val >= 1.81:
                return "Strong"
            
            # 4. FALLTHROUGH
            # If it failed the above, it's Risky (either unprofitable or unsafe balance sheet)
            else:
                return "Risky"

        # Check if we have cached data for this stock that's not expired
        cached_data = cache.get(ticker)
        if cached_data and (current_time - cached_data['timestamp'] < expiry_seconds):
            if cached_data.get('roic') == -999: 
                continue  # Skip if previous fetch failed

        # FETCH NEW DATA using yfinance
        try:
            stock = yf.Ticker(ticker)
            fin = stock.financials      # Income statement data
            bs = stock.balance_sheet    # Balance sheet data
            
            # Check if Yahoo actually gave us data
            if fin.empty or bs.empty:
                print(f"   No data for {ticker} (skipping)")
                continue
            
            # --- CALCULATE 4-YEAR AVERAGE OPERATING MARGIN ---
            # We look at multiple years to see if profitability is consistent
            try:
                # Try to get Operating Income (might be called different things)
                if 'Operating Income' in fin.index:
                    op_income_history = fin.loc['Operating Income']
                elif 'EBIT' in fin.index:
                    op_income_history = fin.loc['EBIT']
                else:
                    op_income_history = pd.Series([0])

                # Get Revenue for same periods
                revenue_history = fin.loc['Total Revenue']
                
                # Calculate margin for each year: Operating Income / Revenue
                yearly_margins = (op_income_history / revenue_history).dropna()
                
                if len(yearly_margins) > 0:
                    avg_margin = yearly_margins.mean()  # Average of all years
                    
                    # Check if average meets our thresholds
                    is_fortress_margin = avg_margin > FORTRESS_MARGIN_THRESHOLD
                    is_positive_margin = avg_margin > 0
                else:
                    is_fortress_margin = False
                    is_positive_margin = False

            except Exception as e:
                is_fortress_margin = False
                is_positive_margin = False

            # --- STANDARD CALCULATIONS ---
            def get_item(df, keys):
                """Helper to get values that might have different names."""
                for k in keys:
                    if k in df.index: 
                        return df.loc[k].iloc[0]
                return 0

            # Get needed values
            ebit = get_item(fin, ['EBIT', 'Operating Income', 'Pretax Income'])
            int_exp = get_item(fin, ['Interest Expense', 'Interest Expense Non Operating'])
            total_assets = get_item(bs, ['Total Assets'])
            curr_liab = get_item(bs, ['Current Liabilities', 'Total Current Liabilities'])
            
            # Calculate Interest Coverage Ratio
            # This tells us if the company can pay its interest bills
            int_exp = abs(int_exp)  # Make positive (expenses are often negative)
            if int_exp == 0: 
                int_cov = 100  # No debt = infinite coverage, use 100 as placeholder
            else: 
                int_cov = ebit / int_exp
            
            # Calculate ROIC (Return on Invested Capital)
            # Shows how efficiently management uses capital
            invested_cap = total_assets - curr_liab  # Simplified invested capital
            if invested_cap <= 0: 
                roic = 0
            else: 
                roic = ebit / invested_cap
            
            # Calculate Altman Z-Score
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            mkt_cap_raw = base_row['Mkt Cap (B)'] * 1_000_000_000  # Convert back to dollars
            z = calculate_altman_z_yfinance(bs, fin, mkt_cap_raw)
            
            # Store metrics in cache for future use
            metrics = {
                'timestamp': current_time,
                'z_score': round(z, 2),
                'roic': roic,
                'int_cov': round(int_cov, 2)
            }
            cache[ticker] = metrics
            
            # Determine final tier based on all metrics
            tier = determine_tier_history(metrics, is_fortress_margin, is_positive_margin)

            # Add all data to our final results
            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'P/E': base_row['P/E'],
                'Sector': base_row['Sector'],
                'Z-Score': round(z, 2),
                'ROIC %': round(roic * 100, 2),
                'Op Margin %': base_row['Op Margin %'],
                'Avg Margin (4Y)': round(avg_margin * 100, 2) if 'avg_margin' in locals() else 0,
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': round(int_cov, 2),
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            
        except Exception as e:
            continue  # Skip if any error

    # Save updated cache to file
    save_cache(cache)
    
    return pd.DataFrame(final_data)

In [6]:
# =============================================================================
# CELL 6: MAIN EXECUTION
# =============================================================================
# This is where we actually RUN all the functions we defined above.
# The 'if __name__ == "__main__":' is a Python convention - it means
# "only run this code if this file is being run directly, not imported"

if __name__ == "__main__":
    # STEP 1: Get list of all US stocks
    tickers = get_combined_universe()
    
    if len(tickers) > 0:
        # STEP 2: Apply quick filters
        survivors_df = get_initial_survivors(tickers)
        
        if not survivors_df.empty:
            print(f"\n Step 2 Complete. {len(survivors_df)} stocks passed basic filters.")
            
            # STEP 3: Deep financial analysis
            final_results = get_advanced_metrics(survivors_df)
            
            if not final_results.empty:
                # Sort results: First by Tier (alphabetically), then by Z-Score (highest first)
                final_results = final_results.sort_values(
                    by=['Tier', 'Z-Score'], 
                    ascending=[True, False]  # True = A-Z, False = highest first
                )
                
                # Split into three categories based on tier
                fortress_df = final_results[final_results['Tier'] == 'Fortress'].copy()
                strong_df = final_results[final_results['Tier'] == 'Strong'].copy()
                risky_df = final_results[final_results['Tier'] == 'Risky'].copy()
                
                # Save to CSV files (spreadsheet format)
                try:
                    fortress_df.to_csv(FORTRESS_CSV, index=False)  # index=False means don't save row numbers
                    strong_df.to_csv(STRONG_CSV, index=False)
                    risky_df.to_csv(RISKY_CSV, index=False)
                    
                    print("\n" + "="*60)
                    print("RESULTS GENERATED")
                    print("="*60)
                    print(f"1. FORTRESS ({len(fortress_df)}): Saved to '{FORTRESS_CSV}'")
                    print(f"2. STRONG   ({len(strong_df)}): Saved to '{STRONG_CSV}'")
                    print(f"3. RISKY    ({len(risky_df)}): Saved to '{RISKY_CSV}'")
                    
                except Exception as e:
                    print(f"\n  Error Saving Files: {e}")
                
                # Set display options for pandas (so we can see more data)
                pd.set_option('display.max_rows', 500)
                pd.set_option('display.max_columns', 20)
                pd.set_option('display.width', 1000)
                
                # Show preview of fortress stocks
                print("\n--- FORTRESS PREVIEW ---")
                print(fortress_df.head(15))  # .head(15) shows first 15 rows
            else:
                print("No stocks passed the deep financial analysis.")
        else:
            print("No stocks passed the initial lightweight filter.")
    else:
        print("Could not fetch ticker universe.")

--- STEP 1: Fetching North American Universe ---
   -> Found 6008 US stocks.

--- STEP 2: Running 'Lightweight' Filter on 6008 stocks ---
 -> Processing Batch 1/13...
 -> Processing Batch 6/13...
 -> Processing Batch 11/13...

 Step 2 Complete. 553 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 553 Survivors ---
 -> Analyzing 1/553: A...
 -> Analyzing 21/553: ALGM...
 -> Analyzing 41/553: AQN...
 -> Analyzing 61/553: AXL...
 -> Analyzing 81/553: BMY...
 -> Analyzing 101/553: CDE...
 -> Analyzing 121/553: COO...
 -> Analyzing 141/553: CYH...
 -> Analyzing 161/553: DUOL...
 -> Analyzing 181/553: ETSY...
 -> Analyzing 201/553: FOUR...
 -> Analyzing 221/553: GLOB...
 -> Analyzing 241/553: HALO...
 -> Analyzing 261/553: HUN...
 -> Analyzing 281/553: ITRG...
 -> Analyzing 301/553: LFST...
 -> Analyzing 321/553: MHK...
 -> Analyzing 341/553: NEM...
 -> Analyzing 361/553: OKTA...
 -> Analyzing 381/553: PGNY...
 -> Analyzing 401/553: QXO...
 -> Analyzing 421/553: RXO...


In [7]:
# =============================================================================
# CELL 7: STEP 4 - ANALYST RATINGS FILTER
# =============================================================================
# This adds another layer of analysis: what do professional analysts think?
# Analyst Rating Scale (typically):
#   1.0 = Strong Buy
#   2.0 = Buy  
#   3.0 = Hold
#   4.0 = Sell
#   5.0 = Strong Sell

def get_analyst_fortress_from_var(df_input):
    """
    Fetch analyst ratings for stocks and filter for those rated "Buy" or better.
    
    Args:
        df_input: DataFrame of stocks to analyze (usually fortress_df)
        
    Returns:
        DataFrame: Stocks with analyst ratings and upside potential
    """
    working_df = df_input.copy()
    tickers = working_df['Ticker'].tolist()
    
    print(f"\n--- STEP 4: Fetching Analyst Ratings for {len(tickers)} Stocks ---")
    print("    (Fetching serially to avoid throttling...)")
    
    analyst_data = []
    
    for i, ticker in enumerate(tickers):
        # Print progress every 10 stocks
        if i % 10 == 0: 
            print(f" -> Analyst Scan {i+1}/{len(tickers)}: {ticker}...")
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info  # Get all available info for the stock
            
            # Extract analyst recommendation (1.0 to 5.0 scale)
            rec_mean = info.get('recommendationMean')
            target_price = info.get('targetMeanPrice')  # Average price target
            current_price = info.get('currentPrice')
            
            # Filter: Only keep stocks rated 2.5 or better (Buy or Strong Buy)
            if rec_mean is None or rec_mean > 2.5: 
                continue
            
            # Calculate upside potential
            # Upside = (Target - Current) / Current * 100
            upside = 0
            if target_price and current_price:
                upside = round(((target_price - current_price) / current_price) * 100, 2)
            
            # Get the original data and add new columns
            base_row = working_df[working_df['Ticker'] == ticker].iloc[0].to_dict()
            base_row['Analyst_Rating'] = rec_mean
            base_row['Target_Price'] = target_price
            base_row['Upside_%'] = upside
            
            analyst_data.append(base_row)
            time.sleep(0.2)  # Small delay to not be timed out of Yahoo's servers
            
        except Exception:
            continue
            
    return pd.DataFrame(analyst_data)

# ==========================================
# EXECUTE THE ANALYST FILTER
# ==========================================

# Check if fortress_df exists from the previous step
if 'fortress_df' in locals() and not fortress_df.empty:
    
    # Run the function
    Analyst_Fortress_DF = get_analyst_fortress_from_var(fortress_df)
    
    if not Analyst_Fortress_DF.empty:
        # Sort by highest upside potential
        Analyst_Fortress_DF = Analyst_Fortress_DF.sort_values(by='Upside_%', ascending=False)
        
        print("\n  Analyst Scan Complete!")
        print(f"Found {len(Analyst_Fortress_DF)} stocks with Buy Ratings (Score < 2.5)")
        
        # Save to CSV
        Analyst_Fortress_DF.to_csv(ANALYST_CSV, index=False)
        print(f"Saved to '{ANALYST_CSV}'")
        
        # Show top picks
        cols = ['Ticker', 'Price', 'Analyst_Rating', 'Upside_%', 'Target_Price', 'Tier']
        print(Analyst_Fortress_DF[cols].head(20))
    else:
        print("No stocks passed the Analyst filter.")
else:
    print("  'fortress_df' not found. Please run Steps 1-3 first.")


--- STEP 4: Fetching Analyst Ratings for 218 Stocks ---
    (Fetching serially to avoid throttling...)
 -> Analyst Scan 1/218: WPM...
 -> Analyst Scan 11/218: DOCS...
 -> Analyst Scan 21/218: CPRX...
 -> Analyst Scan 31/218: EXEL...
 -> Analyst Scan 41/218: PGNY...
 -> Analyst Scan 51/218: RVLV...
 -> Analyst Scan 61/218: LULU...
 -> Analyst Scan 71/218: CTSH...
 -> Analyst Scan 81/218: ALKS...
 -> Analyst Scan 91/218: TS...
 -> Analyst Scan 101/218: FERG...
 -> Analyst Scan 111/218: ECL...
 -> Analyst Scan 121/218: LRN...
 -> Analyst Scan 131/218: VLO...
 -> Analyst Scan 141/218: MWA...
 -> Analyst Scan 151/218: GNRC...
 -> Analyst Scan 161/218: B...
 -> Analyst Scan 171/218: TMHC...
 -> Analyst Scan 181/218: IR...
 -> Analyst Scan 191/218: CROX...
 -> Analyst Scan 201/218: ALC...
 -> Analyst Scan 211/218: IMAX...

  Analyst Scan Complete!
Found 186 stocks with Buy Ratings (Score < 2.5)
Saved to 'YfinanceDataDump\Analyst_Fortress_Picks.csv'
    Ticker    Price  Analyst_Rating  Upside

In [8]:
# =============================================================================
# CELL 8: BUFFETT VALUE SCAN
# =============================================================================
# This filter looks for stocks Warren Buffett might like:
# - Trading BELOW book value (P/B < 1.0) means you're buying $1 of assets for less than $1
# - Positive Return on Equity (ROE) shows the company is profitable
# - Reasonable debt levels (Debt/Equity < 100%)

def get_buffett_value_picks(df_input):
    """
    Find deep value stocks trading below their book value.
    
    Book Value = Total Assets - Total Liabilities
    P/B Ratio = Stock Price / Book Value per Share
    
    If P/B < 1.0, you're theoretically buying the company for less than
    what it would be worth if you sold all its assets and paid all debts.
    
    Args:
        df_input: DataFrame of stocks to analyze
        
    Returns:
        DataFrame: Deep value stocks sorted by P/B ratio
    """
    print(f"\n--- STEP 5: Warren Buffett 'Below NAV' Scan ---")
    print(f"    Scanning {len(df_input)} candidates for Deep Value...")
    print("    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%")

    tickers = df_input['Ticker'].tolist()
    buffett_candidates = []

    # Process in chunks for speed
    chunk_size = 250
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            # Get key statistics and financial data
            data = yq.get_modules("defaultKeyStatistics financialData")

            for symbol in chunk:
                if isinstance(data, dict) and symbol in data:
                    try:
                        stats = data[symbol].get('defaultKeyStatistics', {})
                        fin = data[symbol].get('financialData', {})

                        # 1. Price to Book < 1.0 (The Core "Value" Rule)
                        pb = stats.get('priceToBook')
                        if pb is None or pb >= 1.0 or pb <= 0: 
                            continue

                        # 2. Positive ROE (Return on Equity - company makes money)
                        roe = fin.get('returnOnEquity', 0)
                        if roe is None or roe <= 0: 
                            continue

                        # 3. Reasonable Debt (not overleveraged)
                        de = fin.get('debtToEquity', 0)
                        if de is None or de > 100: 
                            continue

                        # Get base data and add value metrics
                        base_row = df_input[df_input['Ticker'] == symbol].iloc[0].to_dict()
                        base_row['P/B Ratio'] = round(pb, 2)
                        base_row['ROE %'] = round(roe * 100, 2)
                        base_row['Debt/Eq %'] = round(de, 2)

                        buffett_candidates.append(base_row)

                    except: 
                        continue
        except: 
            continue

    return pd.DataFrame(buffett_candidates)

# Run the Buffett scan
if 'final_results' in locals() and not final_results.empty:
    Buffett_Value_DF = get_buffett_value_picks(final_results)
    
    if not Buffett_Value_DF.empty:
        Buffett_Value_DF = Buffett_Value_DF.sort_values(by='P/B Ratio', ascending=True)
        Buffett_Value_DF.to_csv(BUFFETT_CSV, index=False)
        
        print("\n" + "="*60)
        print("BUFFETT SCAN COMPLETE")
        print("="*60)
        print(f"Found {len(Buffett_Value_DF)} Deep Value Stocks")
        
        cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Sector', 'Tier']
        print("\n--- DEEP VALUE PICKS ---")
        print(Buffett_Value_DF[cols].head(20))
    else:
        print("\n  No stocks passed the Buffett filter.")
else:
    print(" 'final_results' not found. Please run Steps 1-3 first.")


--- STEP 5: Warren Buffett 'Below NAV' Scan ---
    Scanning 549 candidates for Deep Value...
    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%

BUFFETT SCAN COMPLETE
Found 15 Deep Value Stocks

--- DEEP VALUE PICKS ---
   Ticker     Price  P/B Ratio  ROE %  Debt/Eq %                  Sector      Tier
8    ACHC   11.9100       0.35   3.69      74.49              Healthcare     Risky
0    GMAB   33.8500       0.36  29.41       2.47              Healthcare  Fortress
7      KT   19.3000       0.39   5.08      60.17  Communication Services     Risky
4     SSL    7.1486       0.50   5.07      76.56         Basic Materials     Risky
6    ANGI   12.6700       0.56   3.42      54.03  Communication Services     Risky
3     HLX    7.1800       0.67   2.71      39.52                  Energy     Risky
11     MT   48.4000       0.67   4.72      26.23         Basic Materials    Strong
13   MOMO    6.9500       0.68   6.72       1.43  Communication Services    Strong
12  

In [9]:
# =============================================================================
# CELL 9: INSIDER TRADING FILTER For Fortress Stocks
# =============================================================================
# Insiders (executives, directors, major shareholders) sometimes have better
# information about their company. When they BUY their own stock, it can be
# a positive sign. When they SELL, it might be neutral (paying taxes) or negative.

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def filter_for_insider_buying(tickers):
    """
    Find stocks where company insiders are NET BUYERS.
    
    Net Buying = Total shares bought - Total shares sold
    If positive, insiders are accumulating shares (bullish signal).
    
    Args:
        tickers: List of stock symbols to check
        
    Returns:
        DataFrame: Stocks with positive insider buying
    """
    print(f" Scanning {len(tickers)} stocks for Insider Buying...")
    insider_picks = []
    
    # Process in smaller chunks to avoid timeout
    chunk_size = 25
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            
            # Fetch insider transaction data
            df_insiders = yq.insider_transactions
            
            # Fetch current prices
            price_data = yq.price
            
            # Skip if no data
            if isinstance(df_insiders, dict) or not hasattr(df_insiders, 'reset_index'):
                continue
            
            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders['symbol'].values:
                    continue
                
                # Get transactions for this stock
                stock_tx = df_insiders[df_insiders['symbol'] == symbol].copy()
                
                # Separate purchases and sales
                # The text typically says "Purchase" or "Sale"
                buys = stock_tx[stock_tx['transactionText'].astype(str).str.contains(
                    "Purchase", case=False, na=False)]
                sells = stock_tx[stock_tx['transactionText'].astype(str).str.contains(
                    "Sale", case=False, na=False)]
                
                # Calculate total buy/sell volume
                buy_vol = buys['shares'].sum() if not buys.empty else 0
                sell_vol = sells['shares'].sum() if not sells.empty else 0
                
                # Get current price
                current_price = None
                try:
                    if isinstance(price_data, dict) and symbol in price_data:
                        current_price = price_data[symbol].get('regularMarketPrice', None)
                except:
                    current_price = None

                # Only keep if net buying is positive
                if buy_vol > sell_vol:
                    insider_picks.append({
                        'Ticker': symbol,
                        'Current_Price': current_price,
                        'Insider_Buys_Count': len(buys),
                        'Net_Shares_Bought': buy_vol - sell_vol
                    })
                        
        except Exception as e:
            continue

    return pd.DataFrame(insider_picks)

# Run the insider filter
if 'fortress_df' in locals() and not fortress_df.empty:
    target_tickers = fortress_df['Ticker'].tolist()
else:
    print("'fortress_df' not found.")

Fortress_insiders = filter_for_insider_buying(target_tickers)
print(f"Created 'Fortress_insiders' with {len(Fortress_insiders)} rows.")
display(Fortress_insiders)

 Scanning 218 stocks for Insider Buying...
Created 'Fortress_insiders' with 12 rows.


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought
0,HWM,224.26,3,167433.0
1,AGI,40.30,21,1650770.0
2,CDE,21.22,6,14327.0
3,GIL,63.81,105,1426104.0
4,SHLS,9.37,3,8335.0
5,CGAU,16.06,88,4428018.0
6,RAL,53.91,1,2000.0
7,TMO,624.65,10,3557415.0
8,FSM,10.27,10,2116207.0
9,OPCH,36.11,8,18954.0


In [10]:

# ==========================================
# 10. ANALYST FILTER FUNCTION FOR INSIDER PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(df_insiders, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if df_insiders.empty:
        return df_insiders
        
    tickers = df_insiders['Ticker'].tolist()
    
    
    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data
        
        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and 'recommendationMean' in data:
                    score = data.get('recommendationMean')
                    
                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append({
                            'Ticker': t,
                            'Analyst_Score': score,
                            'Analyst_Verdict': data.get('recommendationKey', 'N/A')
                        })
        
        df_analyst = pd.DataFrame(analyst_data)
        
        if df_analyst.empty:
            print(" No Analyst ratings found for these tickers.")
            return df_insiders # Return original if no data found
            
        # Merge with the Insider DataFrame
        merged = pd.merge(df_insiders, df_analyst, on='Ticker', how='inner')
        
        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged['Analyst_Score'] <= max_score].copy()
        
        print(f"  Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy).")
        return final_df.sort_values(by='Analyst_Score', ascending=True)

    except Exception as e:
        print(f"  Error in Analyst Filter: {e}")
        return df_insiders

# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if 'fortress_df' in locals() and not fortress_df.empty:
    target_tickers = fortress_df['Ticker'].tolist()
else:
    # Backup list just in case need to update periodically
    target_tickers = [
        'PET.TO', 'MFI.TO', 'TXG.TO', 'SAP.TO', 'PAAS.TO', 'NEO.TO', 'WPM.TO', 
        'FNV.TO', 'LUG.TO', 'DPM.TO', 'ASM.TO', 'PNG.V', 'DSG.TO', 'KNT.TO', 
        'GGD.TO', 'GRGD.TO', 'WDO.TO', 'OGC.TO', 'DNG.TO', 'CLS.TO'
    ]

# B. Run Insider Filter
insider_winners = filter_for_insider_buying(target_tickers)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with Data Wrangler
if not insider_winners.empty:
    Fortress_insiders_Analyst_buy = filter_for_analyst_ratings(insider_winners, max_score=2.5)
else:
    Fortress_insiders_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_insiders_Analyst_buy.empty:
    print(f"\n  Final List: {len(Fortress_insiders_Analyst_buy)} stocks (Fortress + Insider Buying + Analyst Buy Rating)")
    display(Fortress_insiders_Analyst_buy)
else:
    print("No stocks passed all filters.")

 Scanning 218 stocks for Insider Buying...
  Analyst Filter: 12 -> 12 stocks (Min Rating: Buy).

  Final List: 12 stocks (Fortress + Insider Buying + Analyst Buy Rating)


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought,Analyst_Score,Analyst_Verdict
10,DHT,13.28,3,413316.0,1.20000,strong_buy
1,AGI,40.30,21,1650770.0,1.38462,strong_buy
0,HWM,224.26,3,167433.0,1.43478,strong_buy
9,OPCH,36.11,8,18954.0,1.50000,strong_buy
3,GIL,63.81,105,1426104.0,1.53846,buy
7,TMO,624.65,10,3557415.0,1.53846,buy
2,CDE,21.22,6,14327.0,1.85714,buy
4,SHLS,9.37,3,8335.0,1.95238,buy
6,RAL,53.91,1,2000.0,2.25000,buy
5,CGAU,16.06,88,4428018.0,2.50000,buy


In [11]:
# =============================================================================
# CELL 11: BURRY EV/EBITDA FILTER For Fortress Stocks
# =============================================================================
# EV/EBITDA is a valuation metric popular with hedge fund manager Michael Burry.
# 
# EV = Enterprise Value = Market Cap + Debt - Cash
#      (What it would cost to buy the whole company)
# 
# EBITDA = Earnings Before Interest, Taxes, Depreciation, and Amortization
#          (Proxy for operating cash flow)
# 
# EV/EBITDA tells you how many years of cash flow it would take to buy the company.
# Lower is better (cheaper stock).
# We compare each stock to its SECTOR AVERAGE to find relative value.

def filter_burry_ev_ebitda(df_input):
    """
    Find stocks trading at a discount to their sector average.
    
    Logic: A stock with EV/EBITDA of 8x is "cheap" in a sector 
           where the average is 15x.
    
    Args:
        df_input: DataFrame of stocks to analyze (usually fortress_df)
        
    Returns:
        DataFrame: Stocks cheaper than their sector average
    """
    if df_input is None or df_input.empty:
        print(" Input DataFrame is empty.")
        return pd.DataFrame()

    print(f"Analyzing EV/EBITDA for {len(df_input)} Fortress stocks...")
    
    tickers = df_input['Ticker'].tolist()
    
    # Fetch data in bulk
    try:
        yq = Ticker(tickers, asynchronous=True)
        data = yq.get_modules('defaultKeyStatistics financialData summaryDetail')
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

    ev_data = []
    
    for ticker in tickers:
        try:
            ticker_data = data.get(ticker, {})
            if isinstance(ticker_data, str): 
                continue
            
            stats = ticker_data.get('defaultKeyStatistics', {})
            fin_data = ticker_data.get('financialData', {})
            summary = ticker_data.get('summaryDetail', {})

            # Try to get pre-calculated EV/EBITDA
            ev_ebitda = stats.get('enterpriseToEbitda')
            
            # If not available, calculate manually
            if ev_ebitda is None:
                try:
                    market_cap = summary.get('marketCap')
                    total_debt = fin_data.get('totalDebt')
                    total_cash = fin_data.get('totalCash')
                    ebitda = fin_data.get('ebitda')
                    
                    if all(v is not None for v in [market_cap, total_debt, total_cash, ebitda]):
                        if ebitda != 0:
                            # EV = Market Cap + Debt - Cash
                            enterprise_value = market_cap + total_debt - total_cash
                            ev_ebitda = enterprise_value / ebitda
                except:
                    pass

            # Only include positive values (profitable companies)
            if ev_ebitda is not None and ev_ebitda > 0:
                ev_data.append({
                    'Ticker': ticker,
                    'EV/EBITDA': round(ev_ebitda, 2)
                })
        except:
            continue
            
    df_vals = pd.DataFrame(ev_data)
    
    if df_vals.empty:
        print("Could not retrieve EV/EBITDA data.")
        return pd.DataFrame()

    # Merge with sector data
    merged_df = pd.merge(df_input, df_vals, on='Ticker', how='inner')
    
    # Calculate sector averages
    print("\n--- SECTOR AVERAGES (EV/EBITDA) ---")
    sector_stats = merged_df.groupby('Sector')['EV/EBITDA'].mean().reset_index()
    sector_stats.rename(columns={'EV/EBITDA': 'Sector_Avg_EV_EBITDA'}, inplace=True)
    sector_stats['Sector_Avg_EV_EBITDA'] = sector_stats['Sector_Avg_EV_EBITDA'].round(2)
    
    print(sector_stats.to_string(index=False))
    
    # Merge with sector averages
    final_df = pd.merge(merged_df, sector_stats, on='Sector', how='left')
    
    # Filter: Keep only stocks CHEAPER than sector average
    burry_picks = final_df[final_df['EV/EBITDA'] < final_df['Sector_Avg_EV_EBITDA']].copy()
    
    # Calculate discount percentage
    # Discount = 1 - (Stock EV/EBITDA / Sector Average)
    burry_picks['Discount_%'] = round(
        (1 - (burry_picks['EV/EBITDA'] / burry_picks['Sector_Avg_EV_EBITDA'])) * 100, 2
    )
    
    # Sort by biggest discount
    burry_picks = burry_picks.sort_values(by='Discount_%', ascending=False)
    
    return burry_picks

# Run the Burry filter
if 'fortress_df' in locals() and not fortress_df.empty:
    Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)
    
    if not Fortress_Burry_EV_EBITDA.empty:
        print(f"\n Found {len(Fortress_Burry_EV_EBITDA)} Undervalued Stocks")
        display(Fortress_Burry_EV_EBITDA[['Ticker', 'Sector', 'Price', 'EV/EBITDA', 
                                          'Sector_Avg_EV_EBITDA', 'Discount_%', 'Tier']])

Analyzing EV/EBITDA for 218 Fortress stocks...

--- SECTOR AVERAGES (EV/EBITDA) ---
                Sector  Sector_Avg_EV_EBITDA
       Basic Materials                 15.81
Communication Services                 19.61
     Consumer Cyclical                 14.21
    Consumer Defensive                 15.86
                Energy                  9.42
            Healthcare                 21.32
           Industrials                 20.27
            Technology                 21.58

 Found 129 Undervalued Stocks


,Ticker,Sector,Price,EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Tier
181,VSNT,Communication Services,32.6500,1.81,19.61,90.77,Fortress
46,CALM,Consumer Defensive,77.1200,1.52,15.86,90.42,Fortress
162,WLTH,Technology,10.1500,3.07,21.58,85.77,Fortress
203,PDD,Consumer Cyclical,107.4200,2.05,14.21,85.57,Fortress
32,AVGO,Technology,343.0200,4.83,21.58,77.62,Fortress
60,PLAB,Technology,34.2200,5.94,21.58,72.47,Fortress
160,FSM,Basic Materials,10.2799,4.77,15.81,69.83,Fortress
174,PPC,Consumer Defensive,41.6300,5.38,15.86,66.08,Fortress
20,CPRX,Healthcare,22.9600,7.29,21.32,65.81,Fortress
124,CGAU,Basic Materials,16.0700,6.21,15.81,60.72,Fortress


In [12]:

# ==========================================
# 12. ANALYST FILTER FUNCTION FOR Burry EV/EBITDA PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(Fortress_Burry_EV_EBITDA, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if Fortress_Burry_EV_EBITDA.empty:
        return Fortress_Burry_EV_EBITDA
        
    tickers = Fortress_Burry_EV_EBITDA['Ticker'].tolist()
    
    
    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data
        
        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and 'recommendationMean' in data:
                    score = data.get('recommendationMean')
                    
                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append({
                            'Ticker': t,
                            'Analyst_Score': score,
                            'Analyst_Verdict': data.get('recommendationKey', 'N/A')
                        })
        
        df_analyst = pd.DataFrame(analyst_data)
        
        if df_analyst.empty:
            print("No Analyst ratings found for these tickers.")
            return Fortress_Burry_EV_EBITDA # Return original if no data found
            
        # Merge with the Fortress DataFrame
        merged = pd.merge(Fortress_Burry_EV_EBITDA, df_analyst, on='Ticker', how='inner')
        
        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged['Analyst_Score'] <= max_score].copy()
        
        print(f"Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy).")
        return final_df.sort_values(by='Analyst_Score', ascending=True)

    except Exception as e:
        print(f"Error in Analyst Filter: {e}")
        return Fortress_Burry_EV_EBITDA

# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if 'Fortress_Burry_EV_EBITDA' in locals() and not Fortress_Burry_EV_EBITDA.empty:
    target_tickers = Fortress_Burry_EV_EBITDA['Ticker'].tolist()


# B. Run Insider Filter
Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with Data Wrangler
if not Fortress_Burry_EV_EBITDA.empty:
    Fortress_Burry_Analyst_buy = filter_for_analyst_ratings(Fortress_Burry_EV_EBITDA, max_score=2.5)
else:
    Fortress_Burry_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_Burry_Analyst_buy.empty:
    print(f"\nFinal List: {len(Fortress_Burry_Analyst_buy)} stocks (Fortress + Burry + Analyst Buy Rating)")
    display(Fortress_Burry_Analyst_buy)
else:
    print("No stocks passed all filters.")

Analyzing EV/EBITDA for 218 Fortress stocks...

--- SECTOR AVERAGES (EV/EBITDA) ---
                Sector  Sector_Avg_EV_EBITDA
       Basic Materials                 15.81
Communication Services                 19.61
     Consumer Cyclical                 14.21
    Consumer Defensive                 15.86
                Energy                  9.42
            Healthcare                 21.32
           Industrials                 20.27
            Technology                 21.58
Analyst Filter: 126 -> 105 stocks (Min Rating: Buy).

Final List: 105 stocks (Fortress + Burry + Analyst Buy Rating)


,Ticker,Tier,Price,P/E,Sector,Z-Score,ROIC %,Op Margin %,Avg Margin (4Y),Curr Ratio,Int Cov,Mkt Cap (B),EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Analyst_Score,Analyst_Verdict
8,CPRX,Fortress,22.9600,13.43,Healthcare,16.18,26.70,44.66,36.56,6.622,100.00,2.82,7.29,21.32,65.81,1.00000,strong_buy
5,PLAB,Fortress,34.2200,15.01,Technology,7.81,13.54,24.15,26.03,5.366,4033.44,2.02,5.94,21.58,72.47,1.00000,strong_buy
11,HAFN,Fortress,5.8250,9.55,Industrials,3.08,27.25,16.77,24.63,1.476,15.86,2.92,8.03,20.27,60.38,1.00000,strong_buy
106,DHT,Fortress,13.2750,10.71,Energy,3.63,15.43,37.76,18.35,2.411,6.99,2.13,8.42,9.42,10.62,1.20000,strong_buy
62,Q,Fortress,93.0500,35.65,Technology,9.15,7.85,21.71,19.13,1.781,3.21,19.49,14.97,21.58,30.63,1.22222,strong_buy
57,SGHC,Fortress,10.0200,23.86,Consumer Cyclical,8.57,31.26,23.52,10.14,1.750,32.04,5.07,9.40,14.21,33.85,1.25000,strong_buy
120,MSFT,Fortress,456.6600,32.48,Technology,9.15,26.37,48.87,43.52,1.401,52.84,3394.43,20.62,21.58,4.45,1.28070,strong_buy
4,AVGO,Fortress,343.0200,72.06,Technology,11.91,17.00,31.77,39.71,1.705,8.08,1626.35,4.83,21.58,77.62,1.30612,strong_buy
67,YUMC,Fortress,48.1800,19.99,Consumer Cyclical,3.86,13.58,12.60,8.82,1.320,100.00,17.08,10.17,14.21,28.43,1.33333,strong_buy
101,SVM,Fortress,10.2000,92.73,Basic Materials,5.70,10.35,42.70,30.35,4.588,22.11,2.25,13.89,15.81,12.14,1.33333,strong_buy


In [13]:
# ==========================================
# 13. THE "DEEP VALUE" INTERSECTION (Buffett + Burry)
# ==========================================

# 1. Ensure we have the Buffett Data
if 'Buffett_Value_DF' not in locals():
    print("Buffett Data not found. Running scan now...")
    if 'get_buffett_value_picks' in globals() and 'final_results' in locals():
        Buffett_Value_DF = get_buffett_value_picks(final_results)
    else:
        print("Missing 'final_results' or 'get_buffett_value_picks' function.")
        Buffett_Value_DF = pd.DataFrame()

# 2. Ensure we have the Burry Data
if 'Fortress_Burry_EV_EBITDA' not in locals():
    print("Please run the Burry EV/EBITDA filter cell first.")
    Fortress_Burry_EV_EBITDA = pd.DataFrame()

# 3. THE MERGE (Finding the Overlap)
if not Buffett_Value_DF.empty and not Fortress_Burry_EV_EBITDA.empty:
    
    # Merge on Ticker to find stocks that appear in BOTH lists
    # We use an 'inner' join, which means "keep only if in both"
    Deep_Value_Intersection = pd.merge(
        Buffett_Value_DF[['Ticker', 'P/B Ratio', 'ROE %', 'Debt/Eq %']], 
        Fortress_Burry_EV_EBITDA[['Ticker', 'Price', 'Sector', 'EV/EBITDA', 'Sector_Avg_EV_EBITDA', 'Discount_%', 'Tier']],
        on='Ticker', 
        how='inner'
    )
    
    if not Deep_Value_Intersection.empty:
        print("\n" + "="*60)
        print(f"DEEP VALUE GEMS FOUND: {len(Deep_Value_Intersection)}")
        print("="*60)
        print("Criteria: Trading < Book Value (Buffett) AND Cheaper than Sector (Burry)")
        
        # Sort by the "Discount" (how cheap they are vs sector)
        Deep_Value_Intersection = Deep_Value_Intersection.sort_values(by='Discount_%', ascending=False)
        
        cols = ['Ticker', 'Price', 'Tier', 'P/B Ratio', 'EV/EBITDA', 'Sector_Avg_EV_EBITDA', 'Discount_%', 'Sector']
        display(Deep_Value_Intersection[cols])
        
    else:
        print("\n No stocks passed BOTH filters.")
        print("This means no stock is both 'Below Book Value' AND 'Cheaper than Sector Average' at the same time.")
        print(f"Buffett Count: {len(Buffett_Value_DF)} | Burry Count: {len(Fortress_Burry_EV_EBITDA)}")

else:
    print("Cannot combine. One of the filters returned 0 results.")


DEEP VALUE GEMS FOUND: 1
Criteria: Trading < Book Value (Buffett) AND Cheaper than Sector (Burry)


,Ticker,Price,Tier,P/B Ratio,EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Sector
0,TGNA,18.99,Fortress,0.98,7.75,19.61,60.48,Communication Services


In [14]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND','ARCC','BANC','ONB','UBER','ADMA','MIR','APG','SEI','FLEX','DD','SVM','WB'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 13 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...

--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
6,GRND,12.42,-2.20,17.46,-50.58,1.40,21.75,0.72,3.17
0,ADMA,17.50,-4.27,17.92,-31.83,1.00,30.00,1.26,3.18
11,UBER,84.38,-0.32,85.39,-17.27,1.47,111.17,0.84,2.40
12,WB,11.01,-0.81,9.98,-15.05,2.00,11.88,0.47,2.53
7,MIR,27.00,2.86,20.30,-10.82,1.12,30.50,0.91,3.51
5,FLEX,66.59,4.32,49.34,-7.80,1.42,75.00,0.75,2.92
2,ARCC,20.99,0.53,20.49,-6.25,1.27,22.64,0.91,1.40
9,SEI,55.40,6.50,33.58,-3.33,1.17,66.27,1.23,5.90
8,ONB,23.29,1.93,21.50,-2.46,1.85,26.23,0.92,2.11
4,DD,43.39,1.17,32.46,-1.77,1.48,47.88,0.86,2.24


In [15]:
tickers_gemini = ['TCL-A.TO'] 
import os
from google import genai
from google.genai import types
from IPython.display import display, Markdown

# ==========================================
# SECURE CONFIGURATION
# ==========================================

# 1. Define the path to your key file
# If the file is in the same folder as this notebook, just use the filename.
KEY_FILE_PATH = r"C:\Users\jdcc3\OneDrive - McMaster University\Gemini API Key\gemini_key.txt"

def load_api_key(filepath):
    """
    Reads the API key from a local file to avoid hardcoding it.
    """
    try:
        with open(filepath, "r") as f:
            # .strip() removes any accidental newlines or spaces
            return f.read().strip()
    except FileNotFoundError:
        print(f"Error: Could not find the file '{filepath}'")
        print("Please create a text file with your API key in it.")
        return None
    except Exception as e:
        print(f"Error reading key file: {e}")
        return None

# 2. Load the key and set the environment variable
api_key = load_api_key(KEY_FILE_PATH)

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key
    print("API Key loaded securely.")
else:
    print("CRITICAL: API Key not loaded. The script will fail.")

# ==========================================
# SENTIMENT ANALYSIS FUNCTION
# ==========================================
def analyze_sentiment_gemini_3(tickers_gemini, company_name=None):
    
    if not os.environ.get("GEMINI_API_KEY"):
        print("Stop: No API Key found.")
        return

    print(f"\nGemini 3 is thinking (High Reasoning Mode)... analyzing ${tickers_gemini}...")

    # Initialize Client
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts=False, 
            thinking_level="HIGH"
        ),
        tools=[types.Tool(
            google_search=types.GoogleSearch() 
        )],
        response_modalities=["TEXT"]
    )

    prompt = f"""
    You are a Senior Equity Research Analyst using the Gemini 3 Reasoning Engine. 
    Perform a deep "Market Sentiment Analysis" on {tickers_gemini} ({company_name if company_name else 'the company'}).
    
    Step 1: SEARCH. Use Google Search to find the latest (last 30 days) news, analyst notes, and SEC filings.
    Step 2: REASON. Analyze the search results to determine the true market psychology. Look for contradictions between price action and news.
    
    Investigate these 4 Pillars:
    1. **News Virality**: Are headlines fear-mongering or euphoric? (Look for scandals, lawsuits, or product breakthroughs).
    2. **Analyst Shifts**: Are price targets moving UP or DOWN in the last week?
    3. **Institutional Flows**: Any reports of hedge funds or insiders buying/selling?
    4. **The "Whisper" Number**: What are traders saying on forums vs. official guidance?

    **OUTPUT FORMAT:**
    Produce a professional Markdown report:
    
    ## Gemini 3 Sentiment Report: {tickers_gemini}
    **Reasoning Depth:** High
    **Sentiment Score:** [1-10]
    **Verdict:** [Buy / Hold / Sell / Speculative]
    
    ### 1. The Bull Thesis (Why it goes up)
    * ...
    
    ### 2. The Bear Thesis (Why it goes down)
    * ...
    
    ### 3. Deep Dive Analysis
    * **News Analysis**: ...
    * **Smart Money**: ...
    * **Financial Statement Analysis**: (Historic performance over last 3 years + expected performance)
    
    ### 4. Conclusion
    [Summary of whether the current price is a trap or an opportunity]
    """

    try:
        response = client.models.generate_content(
            model='gemini-3-flash-preview', # Or 'gemini-3-flash-preview'
            contents=prompt,
            config=config
        )
        display(Markdown(response.text))
        
    except Exception as e:
        print(f"Error: {e}")

# ==========================================
# EXECUTION
# ==========================================
# Only run this if the key loaded successfully
if os.environ.get("GEMINI_API_KEY"):
    analyze_sentiment_gemini_3(tickers_gemini, tickers_gemini)

API Key loaded securely.

Gemini 3 is thinking (High Reasoning Mode)... analyzing $['TCL-A.TO']...


## Gemini 3 Sentiment Report: Transcontinental Inc. (TCL-A.TO)
**Reasoning Depth:** High
**Sentiment Score:** 9.2/10
**Verdict:** Strong Buy (Special Situation / Capital Return Play)

---

### 1. The Bull Thesis (Why it goes up)
*   **Massive Capital Return:** On December 8, 2025, Transcontinental announced the sale of its Packaging Sector for **$2.22 billion CAD**. Most significantly, management intends to distribute **~$20.00 per share** in cash to shareholders upon closing (expected Q1 2026).
*   **The "Free Stub" Opportunity:** With the stock currently trading near **$22.92**, the market is implicitly valuing the remaining business—Retail Services, Printing, and Media—at just **$2.92 per share**. This remaining "stub" generated ~$215M in LTM Adjusted EBITDA, suggesting the market is valuing a stable (albeit low-growth) cash-flow engine at a distressed multiple (~1.5x EBITDA).
*   **Insider & Largest Shareholder Alignment:** The Marcoux family (via Capinabel Inc.), which controls ~66% of voting rights, has already signed a support agreement. Head of Corporate Development Sam Bendavid was also seen buying shares at the $22.50 level in January 2026.

### 2. The Bear Thesis (Why it goes down)
*   **Growth Engine Divested:** The Packaging division was the primary growth driver for the last decade. By selling it, Transcontinental reverts to being a legacy print and flyer business, sectors facing secular decline.
*   **Execution Risk & Regulatory Hurdles:** While highly likely to close, any delay or block by regulators on the $2.1B deal would lead to a catastrophic re-pricing of the stock back to the $15-$17 pre-announcement range.
*   **Operational Headwinds:** The Retail Services sector recently faced a "challenging" Q4 due to Canada Post labor conflicts, which impacted flyer distribution and squeezed short-term margins.

---

### 3. Deep Dive Analysis

#### **News Analysis**
HEADLINE VIRALITY: **High/Euphoric**. 
The narrative shifted instantly in early December from "struggling legacy printer" to "massive cash cow." Headlines are dominated by the $20/share payout. There is no major fear-mongering; even the Canada Post strike news was overshadowed by the sheer scale of the divestiture proceeds. The market is treating this as a "liquidation lite" event where shareholders get their principal back in cash while retaining a piece of the remaining profitable assets.

#### **Smart Money (Analyst & Institutional)**
ANALYST SHIFTS: **Aggressive Upward Revisions**.
*   **RBC Capital:** Raised target from $26 to **$29**.
*   **National Bank:** Raised target from $24 to **$28**.
*   **TD Securities:** Raised target from $27 to **$28**.
*   **BMO Capital:** Raised target from $23 to **$27**.
The consensus target of **$27.33** suggests a ~19% upside from current levels *excluding* the psychology of the $20 payout. Institutional flows remain steady, with major holders like Vanguard and BlackRock maintaining positions, while boutique value funds are likely sniffing out the arbitrage in the stub value.

#### **Financial Statement Analysis**
*   **Historic (Last 3 Years):** Revenue has been flat to slightly declining ($2.7B - $2.8B range). However, net earnings saw a massive 41% jump in 2025 as the company focused on profitability and cost-cutting ($171M net income vs $121M in 2024).
*   **Expected (Pro Forma):** Post-sale, the "New TC" will be debt-light (target 1.7x net debt/EBITDA) and highly focused on the Printing and Media sectors. While revenue will drop by nearly 50% post-divestiture, the remaining entity is expected to be a high-yield, free-cash-flow machine.

#### **The "Whisper" Number**
Official guidance points to a $20 payout. However, the "whisper" among sophisticated traders on forums like *Globe Investor* and *Stockhouse* is that the **actual tax-efficient return of capital** ($7/share) plus the dividend portion (~$13/share) makes this a superior total-return play for taxable accounts in Canada. The "whisper" valuation for the remaining printing business is closer to **$6-$8 per share**, suggesting the stock should fundamentally trade north of **$26-$28** as the deal approaches closing.

---

### 4. Conclusion
The current price of **$22.92** is an **opportunity, not a trap**. 

The market has been slow to fully price in the $20 cash distribution, likely due to the "legacy" stigma attached to the remaining printing business. However, getting paid $20 to hold a stock that will still own a market-leading (and profitable) printing and educational publishing business is a rare arbitrage. As the special meeting concludes in late January 2026 and regulatory approvals clear, we expect the stock to converge toward the **$27-$28** analyst consensus. 

**Investment Action:** Accumulate prior to the special shareholder meeting in late January to lock in the record date for the $20 distribution.